In [29]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Bidirectional, Conv1D
from tensorflow.keras.optimizers import RMSprop, Adam
import spacy
from spacy_langdetect import LanguageDetector
import numpy as np
import random
import sys
import io
import requests
import re
import pandas
from tensorflow.keras import layers 
from tensorflow.keras import callbacks 
from langdetect import detect_langs


In [2]:
books=pandas.read_csv('C:/Users/benoi/Documents/HEC/_Hiver 2021/Machine Learning/projet/final_genres.txt', sep=',',)

#with open('C:/Users/benoi/Documents/HEC/_Hiver 2021/Données textuelles/last_words.txt', 'r') as file:
 #   last_words = file.read()

    
    
books=books.drop(['Unnamed: 0', 'img', 'book_authors'], axis=1)
books = books[books['book_desc'].notnull()]
books = books[books['book_title'].notnull()]
books

,book_desc,book_title,genres,main_label
0,Winning will make you famous. Losing means cer...,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,Young Adult
1,There is a door at the end of a silent corrido...,Harry Potter and the Order of the Phoenix,Fantasy|Young Adult|Fiction,Fantasy
2,The unforgettable novel of a childhood in a sl...,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...,Historical
3,«È cosa ormai risaputa che a uno scapolo in po...,Pride and Prejudice,Classics|Fiction|Romance,Romance
4,About three things I was absolutely positive.F...,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...,Young Adult
...,...,...,...,...
43843,"Translated from the French by Georgina Kleege,...",Sitt Marie Rose,Fiction|Cultural|Lebanon|War|Academic|School|L...,Politics & History
43844,Thirteen-year-old Jasira wants what every girl...,Towelhead,Fiction|Young Adult|Young Adult|Coming Of Age|...,Young Adult
43845,Avi Steinberg is stumped. After defecting from...,Running the Books: The Adventures of an Accide...,Nonfiction|Autobiography|Memoir|Biography|Writ...,Biography
43846,"Soon to be a major motion picture, from Brad P...",He Wanted the Moon: The Madness and Medical Ge...,Nonfiction|Autobiography|Memoir|Biography|Psyc...,Biography


In [3]:
i=0
cd=[]
ct=[]
while i<len(books):
    a=re.sub('[?,!@#*$.«)(]', '', books.iloc[i,0].lower())
    cd.append(a)
    ct.append(re.sub('[?,!@#*$.«)(]', '', books.iloc[i,1].lower()))
    
    i+=1
    
    
books['desc_clean']=cd
books['title_clean']=ct
books



,book_desc,book_title,genres,main_label,desc_clean,title_clean
0,Winning will make you famous. Losing means cer...,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,Young Adult,winning will make you famous losing means cert...,the hunger games
1,There is a door at the end of a silent corrido...,Harry Potter and the Order of the Phoenix,Fantasy|Young Adult|Fiction,Fantasy,there is a door at the end of a silent corrido...,harry potter and the order of the phoenix
2,The unforgettable novel of a childhood in a sl...,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...,Historical,the unforgettable novel of a childhood in a sl...,to kill a mockingbird
3,«È cosa ormai risaputa che a uno scapolo in po...,Pride and Prejudice,Classics|Fiction|Romance,Romance,è cosa ormai risaputa che a uno scapolo in pos...,pride and prejudice
4,About three things I was absolutely positive.F...,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...,Young Adult,about three things i was absolutely positivefi...,twilight
...,...,...,...,...,...,...
43843,"Translated from the French by Georgina Kleege,...",Sitt Marie Rose,Fiction|Cultural|Lebanon|War|Academic|School|L...,Politics & History,translated from the french by georgina kleege ...,sitt marie rose
43844,Thirteen-year-old Jasira wants what every girl...,Towelhead,Fiction|Young Adult|Young Adult|Coming Of Age|...,Young Adult,thirteen-year-old jasira wants what every girl...,towelhead
43845,Avi Steinberg is stumped. After defecting from...,Running the Books: The Adventures of an Accide...,Nonfiction|Autobiography|Memoir|Biography|Writ...,Biography,avi steinberg is stumped after defecting from ...,running the books: the adventures of an accide...
43846,"Soon to be a major motion picture, from Brad P...",He Wanted the Moon: The Madness and Medical Ge...,Nonfiction|Autobiography|Memoir|Biography|Psyc...,Biography,soon to be a major motion picture from brad pi...,he wanted the moon: the madness and medical ge...


In [4]:
from langdetect import detect_langs
i=0
t=[]
while i<len(books):   
    try:
        a=detect_langs(books.iloc[i,0])
        if str(a[0])[:2]=='en':
            i+=1
        else:
            books=books.drop(books.index[i])
       
    except :
         books=books.drop(books.index[i])
        

books

,book_desc,book_title,genres,main_label,desc_clean,title_clean
0,Winning will make you famous. Losing means cer...,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,Young Adult,winning will make you famous losing means cert...,the hunger games
1,There is a door at the end of a silent corrido...,Harry Potter and the Order of the Phoenix,Fantasy|Young Adult|Fiction,Fantasy,there is a door at the end of a silent corrido...,harry potter and the order of the phoenix
2,The unforgettable novel of a childhood in a sl...,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...,Historical,the unforgettable novel of a childhood in a sl...,to kill a mockingbird
4,About three things I was absolutely positive.F...,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...,Young Adult,about three things i was absolutely positivefi...,twilight
5,Trying to make sense of the horrors of World W...,The Book Thief,Historical|Historical Fiction|Fiction|Young Adult,Historical,trying to make sense of the horrors of world w...,the book thief
...,...,...,...,...,...,...
43843,"Translated from the French by Georgina Kleege,...",Sitt Marie Rose,Fiction|Cultural|Lebanon|War|Academic|School|L...,Politics & History,translated from the french by georgina kleege ...,sitt marie rose
43844,Thirteen-year-old Jasira wants what every girl...,Towelhead,Fiction|Young Adult|Young Adult|Coming Of Age|...,Young Adult,thirteen-year-old jasira wants what every girl...,towelhead
43845,Avi Steinberg is stumped. After defecting from...,Running the Books: The Adventures of an Accide...,Nonfiction|Autobiography|Memoir|Biography|Writ...,Biography,avi steinberg is stumped after defecting from ...,running the books: the adventures of an accide...
43846,"Soon to be a major motion picture, from Brad P...",He Wanted the Moon: The Madness and Medical Ge...,Nonfiction|Autobiography|Memoir|Biography|Psyc...,Biography,soon to be a major motion picture from brad pi...,he wanted the moon: the madness and medical ge...


In [5]:
i=0
b=[]
c=[]
g=[]
l=[]
while i<len(books):
    a=books.iloc[i,5].split(' ')
    l.append(a)
    g.append(books.iloc[i,3])
    j=0
    while j<len(a):
        b.append(a[j])
        j+=1
    c.append(len(a))
    i+=1
w=set(b)
dg=set(g)
books['title_list']=l
vocab=len(w)
print(vocab, max(c), len(g), len(b))
books

23138 41 38610 154321


,book_desc,book_title,genres,main_label,desc_clean,title_clean,title_list
0,Winning will make you famous. Losing means cer...,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,Young Adult,winning will make you famous losing means cert...,the hunger games,"[the, hunger, games]"
1,There is a door at the end of a silent corrido...,Harry Potter and the Order of the Phoenix,Fantasy|Young Adult|Fiction,Fantasy,there is a door at the end of a silent corrido...,harry potter and the order of the phoenix,"[harry, potter, and, the, order, of, the, phoe..."
2,The unforgettable novel of a childhood in a sl...,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...,Historical,the unforgettable novel of a childhood in a sl...,to kill a mockingbird,"[to, kill, a, mockingbird]"
4,About three things I was absolutely positive.F...,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...,Young Adult,about three things i was absolutely positivefi...,twilight,[twilight]
5,Trying to make sense of the horrors of World W...,The Book Thief,Historical|Historical Fiction|Fiction|Young Adult,Historical,trying to make sense of the horrors of world w...,the book thief,"[the, book, thief]"
...,...,...,...,...,...,...,...
43843,"Translated from the French by Georgina Kleege,...",Sitt Marie Rose,Fiction|Cultural|Lebanon|War|Academic|School|L...,Politics & History,translated from the french by georgina kleege ...,sitt marie rose,"[sitt, marie, rose]"
43844,Thirteen-year-old Jasira wants what every girl...,Towelhead,Fiction|Young Adult|Young Adult|Coming Of Age|...,Young Adult,thirteen-year-old jasira wants what every girl...,towelhead,[towelhead]
43845,Avi Steinberg is stumped. After defecting from...,Running the Books: The Adventures of an Accide...,Nonfiction|Autobiography|Memoir|Biography|Writ...,Biography,avi steinberg is stumped after defecting from ...,running the books: the adventures of an accide...,"[running, the, books:, the, adventures, of, an..."
43846,"Soon to be a major motion picture, from Brad P...",He Wanted the Moon: The Madness and Medical Ge...,Nonfiction|Autobiography|Memoir|Biography|Psyc...,Biography,soon to be a major motion picture from brad pi...,he wanted the moon: the madness and medical ge...,"[he, wanted, the, moon:, the, madness, and, me..."


In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import collections
from numpy import array
from tensorflow.keras.utils import to_categorical


maxwords=max(c)
nbwords = collections.Counter(c)
print("Frequency of the elements in the List : ",nbwords)

#transformer x
x = books['title_list']
xtoken = Tokenizer()
xtoken.fit_on_texts(x)
x = xtoken.texts_to_sequences(x)
x = pad_sequences(x, maxlen=maxwords, padding='pre')

#transformer y
y = array(books['main_label'])
le = LabelEncoder()

y=le.fit_transform(y)
y = to_categorical(y)

lab2idx = dict(zip(le.classes_, range(len(le.classes_))))
idx2lab = {}
for k, v in lab2idx.items():
    idx2lab[v] = k

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=1)

print (x_train.shape, x_test.shape, y_train.shape, y_test.shape, vocab, idx2lab)
print(x[0])

(25868, 41) (12742, 41) (25868, 13) (12742, 13) 23138 {0: 'Biography', 1: 'Childrens', 2: 'Fantasy', 3: 'Historical', 4: 'Horror', 5: 'Mystery & Crime', 6: 'Poetry', 7: 'Politics & History', 8: 'Religion & Philosophy', 9: 'Romance', 10: 'Science Fiction', 11: 'Sequential Art', 12: 'Young Adult'}
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   1 878 357]


In [10]:
from sklearn.utils import class_weight
import numpy as np
   

#cw = class_weight.compute_class_weight('balanced',np.unique(books['main_label']), books['main_label']),


y_integers = np.argmax(y, axis=1)
cw = class_weight.compute_class_weight('balanced', np.unique(y_integers), y_integers)
cw = dict(enumerate(cw))

print(cw, books['main_label'].value_counts())
print(len(y_integers))

{0: 1.6211790393013101, 1: 2.0040485829959516, 2: 0.3844162567952369, 3: 0.7638888888888888, 4: 2.5384615384615383, 5: 0.8240843507214206, 6: 3.877284595300261, 7: 1.5714285714285714, 8: 1.164249313994512, 9: 0.5062212374296915, 10: 1.3089466725429704, 11: 2.1444043321299637, 12: 0.7105263157894737} Fantasy                  7726
Romance                  5867
Young Adult              4180
Historical               3888
Mystery & Crime          3604
Religion & Philosophy    2551
Science Fiction          2269
Politics & History       1890
Biography                1832
Childrens                1482
Sequential Art           1385
Horror                   1170
Poetry                    766
Name: main_label, dtype: int64
38610


In [12]:
j=0
embedding_vector = {}
glove = open('C:/Users/benoi/Documents/HEC/_Hiver 2021/Machine Learning/projet/glove.6B.300d.txt', encoding="utf8")
for line in glove:
    a = line.split(' ')
    word = a[0]
    coef = np.array(a[1:],dtype = 'float32')
    embedding_vector[word] = coef
    
embedding_matrix = np.zeros((vocab+1,300))
for word,i in xtoken.word_index.items():
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value
    else:
        j+=1
        
print(embedding_matrix.shape, j, 'Pourcentage de mots non trouvés dans GLOVE: ', j/embedding_matrix.shape[0])

(23139, 300) 7007 Pourcentage de mots non trouvés dans GLOVE:  0.3028220752841523


In [13]:
model = Sequential()
model.add(Embedding(vocab+1, 300, weights = [embedding_matrix], input_length=maxwords, trainable = True))
model.add(layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))#filter et kernel
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling1D(pool_size=2))#poolsize
model.add(Bidirectional(LSTM(100)))#100
model.add(Dropout(0.25))#dropout
model.add(Dense(100,activation = 'relu'))
model.add(Dense(len(dg),activation = 'sigmoid'))
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics = ['accuracy'])
ES= callbacks.EarlyStopping(monitor='loss', patience=4)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 41, 300)           6941700   
_________________________________________________________________
conv1d (Conv1D)              (None, 41, 32)            28832     
_________________________________________________________________
batch_normalization (BatchNo (None, 41, 32)            128       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 20, 32)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               106400    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               2

In [42]:
history = model.fit(x_train,y_train, epochs = 20, validation_data=(x_test, y_test), callbacks=[ES], verbose=1, batch_size=128, class_weight=cw)

Epoch 1/20
203/203 [==============================] - 31s 143ms/step - loss: 0.1640 - accuracy: 0.9205 - val_loss: 3.9269 - val_accuracy: 0.3909
Epoch 2/20
203/203 [==============================] - 31s 151ms/step - loss: 0.1532 - accuracy: 0.9235 - val_loss: 4.0182 - val_accuracy: 0.3989
Epoch 3/20
203/203 [==============================] - 32s 160ms/step - loss: 0.1438 - accuracy: 0.9274 - val_loss: 4.0203 - val_accuracy: 0.3962
Epoch 4/20
203/203 [==============================] - 32s 156ms/step - loss: 0.1474 - accuracy: 0.9269 - val_loss: 4.3354 - val_accuracy: 0.4025
Epoch 5/20
203/203 [==============================] - 32s 160ms/step - loss: 0.1516 - accuracy: 0.9260 - val_loss: 4.3699 - val_accuracy: 0.3834
Epoch 6/20
203/203 [==============================] - 33s 160ms/step - loss: 0.1531 - accuracy: 0.9255 - val_loss: 4.0420 - val_accuracy: 0.3962


In [31]:
modelCNN = Sequential()
modelCNN.add(Embedding(vocab+1, 300, weights = [embedding_matrix], input_length=maxwords, trainable = True))#maxwords
modelCNN.add(layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))#a#<ll
modelCNN.add(layers.BatchNormalization())
modelCNN.add(layers.MaxPooling1D(pool_size=2))#poolsize
modelCNN.add(layers.Flatten())
modelCNN.add(Dense(len(dg),activation = 'sigmoid'))#vs relu
optimizer=Adam(learning_rate=0.0001,epsilon=0.1)#lr
modelCNN.compile(optimizer=optimizer,loss='categorical_crossentropy', metrics = ['accuracy'])
ES= callbacks.EarlyStopping(monitor='loss', patience=3)

print(modelCNN.summary())



Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 41, 300)           6941700   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 41, 32)            28832     
_________________________________________________________________
batch_normalization_4 (Batch (None, 41, 32)            128       
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 20, 32)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 640)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 13)                8333      
Total params: 6,978,993
Trainable params: 6,978,929
Non-trainable params: 64
___________________________________________

In [39]:
optimizer=Adam(learning_rate=0.001,epsilon=0.1)
modelCNN.compile(optimizer='Adam',loss='categorical_crossentropy', metrics = ['accuracy'])

In [41]:
ES5= callbacks.EarlyStopping(monitor='loss', patience=5)

historyCNN = modelCNN.fit(x_train,y_train, epochs = 25, validation_data=(x_test, y_test), callbacks=[ES5], verbose=1, batch_size=128, class_weight=cw)

Epoch 1/25
203/203 [==============================] - 25s 117ms/step - loss: 0.2899 - accuracy: 0.8637 - val_loss: 3.8554 - val_accuracy: 0.3869
Epoch 2/25
203/203 [==============================] - 24s 121ms/step - loss: 0.2628 - accuracy: 0.8761 - val_loss: 3.9726 - val_accuracy: 0.3819
Epoch 3/25
203/203 [==============================] - 25s 124ms/step - loss: 0.2412 - accuracy: 0.8856 - val_loss: 4.0350 - val_accuracy: 0.3812
Epoch 4/25
203/203 [==============================] - 25s 125ms/step - loss: 0.2256 - accuracy: 0.8942 - val_loss: 4.0724 - val_accuracy: 0.3867
Epoch 5/25
203/203 [==============================] - 26s 128ms/step - loss: 0.2224 - accuracy: 0.8987 - val_loss: 4.2448 - val_accuracy: 0.3599
Epoch 6/25
203/203 [==============================] - 27s 131ms/step - loss: 0.2168 - accuracy: 0.9001 - val_loss: 4.2482 - val_accuracy: 0.3800
Epoch 7/25
203/203 [==============================] - 27s 133ms/step - loss: 0.2138 - accuracy: 0.9042 - val_loss: 4.3638 - val_ac

[0 0 0 0 0 0 0 1 0 0 0 0 0] [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]


(38613, 41)

In [15]:
def prediction(sentence, modele=model):
    sentence = sentence.lower()
    sentence=sentence.split(' ')
    test = xtoken.texts_to_sequences(sentence)
    flat_list =  [[item for elem in test for item in elem]]
    bob = pad_sequences(flat_list, maxlen=41, padding='pre')
    a=modele.predict(bob)
    b=np.argmax(a, axis=1)
    pred2 = np.argsort(a,axis=1)[:,-2]
    pred3 = np.argsort(a,axis=1)[:,-3]
    genre=idx2lab[b[0]]
    return (genre, a, pred2 pred3)
    



In [24]:
prediction("love murder")

('Romance',
 array([[0.4749476 , 0.01452982, 0.53089595, 0.10134837, 0.8884505 ,
         0.8770132 , 0.04750389, 0.10779369, 0.16176668, 0.9239906 ,
         0.53383493, 0.44093302, 0.8590435 ]], dtype=float32))

In [209]:
# ne pas exécuter
sentence='I love you'
sentence = sentence.lower()
sentence=sentence.split(' ')
test = xtoken.texts_to_sequences(sentence)
flat_list =  [[item for elem in test for item in elem]]
bob = pad_sequences(flat_list, maxlen=41, padding='pre')
print (sentence, test, flat_list, bob)

['i', 'love', 'you'] [[16], [11], [15]] [[16, 11, 15]] [[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0 16 11 15]]


In [176]:
from sklearn.metrics import classification_report
print(classification_report( y_train, predicted_classes, target_names=class_names))

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,   319,     1,   168, 10218,     8,    35,     4,
         197,     5,     3,  6958, 10219])

In [ ]:
model.save('C:/Users/benoi/Documents/HEC/_Hiver 2021/Machine Learning/projet/CNN-LSTM')